In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
best_model = '/kaggle/input/pss5e1-h2o/raw_data_models_final/XGBoost_grid_1_AutoML_1_20250125_200219_model_14'
test = pd.read_csv('/kaggle/input/test-pss5e1/test_pss5e1.csv')
test['time_id'] = test['country'] + '_' + test['product'] + '_' + test['store']
features = [col for col in test.columns if (col!='num_sold' and col!='date' and col!='id')]
cols_to_drop = ['date', 'id']
test.drop(columns=cols_to_drop, inplace=True)

In [32]:
import h2o
h2o.init()
leader_model = h2o.load_model(best_model)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.25" 2024-10-15; OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpe6179yr8
  JVM stdout: /tmp/tmpe6179yr8/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpe6179yr8/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 24 days
H2O_cluster_name:,H2O_from_python_unknownUser_qfqoui
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [33]:
h2o_df = h2o.H2OFrame(test)
preds = leader_model.predict(h2o_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.10/dist-packages/h2o/job.py:81: UserWarning: Test/Validation dataset column 'time_id' has levels not trained on: ["Canada_Holographic Goose_Discount Stickers", "Kenya_Holographic Goose_Discount Stickers"]
  warnings.warn(w)


In [34]:
preds = preds['predict']

In [35]:
from scipy.stats import boxcox
train = pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
train.dropna(subset='num_sold', inplace=True)
vals, lmbda = boxcox(train['num_sold'])

In [36]:
preds = preds.as_data_frame()
preds = preds['predict'].values

/usr/local/lib/python3.10/dist-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [37]:
def inverse_boxcox(y_transformed, lmbda):
    if lmbda==0:
        return np.exp(y_transformed)
    else:
        return np.power(y_transformed*lmbda+1, 1/lmbda)
# preds = inverse_quantile(preds)
preds = inverse_boxcox(preds, lmbda)

In [38]:
sub_test = pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
idx = sub_test['id']

In [39]:
submission_df = pd.DataFrame({
    'id':idx,
    'num_sold':preds
})
submission_df.to_csv('submission.csv', index=False)

In [40]:
h2o.shutdown()

H2O session _sid_8e15 closed.


<ipython-input-40-1edf85295eae>:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


In [41]:
submission_df

,id,num_sold
0,230130,228.188589
1,230131,834.500133
2,230132,739.028282
3,230133,394.688951
4,230134,441.065384
...,...,...
98545,328675,381.703668
98546,328676,2354.246054
98547,328677,1947.085298
98548,328678,1073.785700
